In [ ]:
# Stage 5 | nb42: Event Generation with RAG
# Goal: RAG-driven event generation for Chinese text adventure games

# Cell1:  Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

In [ ]:
# === Cell 2: Import Dependencies ===
import json
import random
import faiss
import numpy as np
from pathlib import Path
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from enum import Enum

In [ ]:
# === Cell 3: Load World KB Index ===
class WorldKB:
    """World Knowledge Base loader for RAG-driven events"""

    def __init__(
        self,
        index_path="indices/world_kb.faiss",
        chunks_path="indices/world_chunks.jsonl",
    ):
        self.embedding_model = SentenceTransformer("BAAI/bge-m3")

        # Load FAISS index
        if Path(index_path).exists():
            self.index = faiss.read_index(index_path)
            print(f"[WorldKB] Loaded index: {self.index.ntotal} chunks")
        else:
            print(f"[WorldKB] Index not found: {index_path}")
            self.index = None
            return

        # Load chunk metadata
        self.chunks = []
        if Path(chunks_path).exists():
            with open(chunks_path, "r", encoding="utf-8") as f:
                for line in f:
                    self.chunks.append(json.loads(line))
            print(f"[WorldKB] Loaded {len(self.chunks)} chunk metadata")

    def search(self, query: str, top_k: int = 5) -> List[Tuple[str, Dict, float]]:
        """Search world KB for relevant context"""
        if self.index is None:
            return []

        # Embed query
        query_vec = self.embedding_model.encode(
            [query], normalize_embeddings=True
        ).astype("float32")

        # Search
        scores, indices = self.index.search(query_vec, top_k)

        results = []
        for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
            if idx < len(self.chunks):
                chunk = self.chunks[idx]
                results.append((chunk["text"], chunk["meta"], float(score)))

        return results


# Load world KB
world_kb = WorldKB()

In [ ]:
# === Cell 4: Event Template System ===
class EventType(Enum):
    EXPLORATION = "exploration"
    ENCOUNTER = "encounter"
    DIALOGUE = "dialogue"
    PUZZLE = "puzzle"
    COMBAT = "combat"
    DISCOVERY = "discovery"


@dataclass
class EventTemplate:
    """Template for generating consistent events"""

    event_type: EventType
    triggers: List[str]  # Conditions that trigger this event
    difficulty_range: Tuple[int, int]  # Min/max difficulty
    knowledge_domains: List[str]  # KB domains to search
    prompt_template: str
    min_choices: int = 2
    max_choices: int = 4


# Define event templates
EVENT_TEMPLATES = {
    EventType.EXPLORATION: EventTemplate(
        event_type=EventType.EXPLORATION,
        triggers=["enter_location", "move_direction"],
        difficulty_range=(1, 3),
        knowledge_domains=["locations", "geography", "environment"],
        prompt_template="""基於以下世界觀資訊，為玩家生成一個探索事件：

世界背景：
{context}

玩家當前狀態：
- 位置：{location}
- 血量：{hp}/{max_hp}
- 物品：{inventory}

請生成一個探索事件，包含：
1. 場景描述（100-150字，氛圍豐富）
2. 2-3個行動選項
3. 每個選項的潛在風險/獎勵提示

風格要求：生動具體，符合世界觀設定，營造適當緊張感。""",
        min_choices=2,
        max_choices=3,
    ),
    EventType.ENCOUNTER: EventTemplate(
        event_type=EventType.ENCOUNTER,
        triggers=["random_encounter", "story_encounter"],
        difficulty_range=(2, 5),
        knowledge_domains=["characters", "creatures", "factions"],
        prompt_template="""基於以下世界觀資訊，為玩家生成一個遭遇事件：

世界背景：
{context}

玩家狀態：
- 位置：{location}
- 血量：{hp}/{max_hp}
- 技能：{skills}
- 物品：{inventory}

生成遭遇事件：
1. 遭遇對象描述（外觀、動機、態度）
2. 情境說明（為什麼遭遇，環境氛圍）
3. 3-4個應對選項（對話、戰鬥、逃跑、其他）
4. 暗示每個選項可能的後果

注意：根據玩家當前血量{hp}調整遭遇難度。""",
        min_choices=3,
        max_choices=4,
    ),
    EventType.DISCOVERY: EventTemplate(
        event_type=EventType.DISCOVERY,
        triggers=["search_action", "investigation"],
        difficulty_range=(1, 4),
        knowledge_domains=["items", "secrets", "lore", "history"],
        prompt_template="""基於世界觀資訊，生成一個發現事件：

相關背景：
{context}

玩家狀態：
- 位置：{location}
- 調查技能：{investigation_skill}
- 現有物品：{inventory}

生成發現事件：
1. 發現的物品/秘密/線索描述
2. 發現的意義和價值
3. 2-3個處理選項
4. 各選項的技能需求和風險

要求：發現內容要與世界觀背景呼應，有深度和意義。""",
    ),
}

In [ ]:
# === Cell 5: RAG-Driven Event Generator ===
class EventGenerator:
    """Core event generation engine with RAG support"""

    def __init__(self, world_kb: WorldKB, model_id: str = "Qwen/Qwen2.5-7B-Instruct"):
        self.world_kb = world_kb
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id, device_map="auto", torch_dtype=torch.float16, load_in_4bit=True
        )

    def generate_event(
        self,
        event_type: EventType,
        player_state: Dict,
        context_query: Optional[str] = None,
    ) -> Dict:
        """Generate event with RAG context"""

        template = EVENT_TEMPLATES.get(event_type)
        if not template:
            raise ValueError(f"No template for event type: {event_type}")

        # Build context query
        if not context_query:
            context_query = f"{player_state.get('location', '')} {event_type.value}"

        # Retrieve relevant world knowledge
        kb_results = self.world_kb.search(context_query, top_k=3)
        context = self._format_context(kb_results)

        # Apply difficulty scaling
        difficulty = self._calculate_difficulty(player_state, template.difficulty_range)

        # Format prompt
        prompt = template.prompt_template.format(
            context=context,
            location=player_state.get("location", "未知區域"),
            hp=player_state.get("hp", 100),
            max_hp=player_state.get("max_hp", 100),
            inventory=", ".join(player_state.get("inventory", [])) or "無",
            skills=", ".join(player_state.get("skills", [])) or "無",
            investigation_skill=player_state.get("investigation", 1),
        )

        # Generate response
        response = self._generate_response(prompt)

        # Extract choices (simplified parsing)
        choices = self._extract_choices(response)

        return {
            "event_id": f"{event_type.value}_{random.randint(1000, 9999)}",
            "type": event_type.value,
            "description": response,
            "choices": choices,
            "difficulty": difficulty,
            "sources": [r[1].get("source_id", "unknown") for r in kb_results],
            "context_used": context[:200] + "..." if len(context) > 200 else context,
        }

    def _format_context(self, kb_results: List[Tuple[str, Dict, float]]) -> str:
        """Format KB search results as context"""
        if not kb_results:
            return "（無相關世界觀資訊）"

        context_parts = []
        for i, (text, meta, score) in enumerate(kb_results[:3]):
            source = meta.get("source_id", f"來源{i+1}")
            context_parts.append(f"[{source}] {text[:150]}")

        return "\n\n".join(context_parts)

    def _calculate_difficulty(
        self, player_state: Dict, range_tuple: Tuple[int, int]
    ) -> int:
        """Calculate event difficulty based on player state"""
        min_diff, max_diff = range_tuple

        # Base difficulty
        difficulty = random.randint(min_diff, max_diff)

        # Adjust based on player HP
        hp_ratio = player_state.get("hp", 100) / player_state.get("max_hp", 100)
        if hp_ratio < 0.3:
            difficulty = max(min_diff, difficulty - 1)  # Easier when low HP
        elif hp_ratio > 0.8:
            difficulty = min(max_diff, difficulty + 1)  # Harder when healthy

        return difficulty

    def _generate_response(self, prompt: str) -> str:
        """Generate text response using LLM"""
        messages = [
            {
                "role": "system",
                "content": "你是一個專業的中文文字冒險遊戲設計師，擅長創造引人入勝的事件和選擇。",
            },
            {"role": "user", "content": prompt},
        ]

        # Format messages
        formatted_prompt = "\n".join([f"{m['role']}: {m['content']}" for m in messages])

        # Tokenize and generate
        inputs = self.tokenizer(
            formatted_prompt, return_tensors="pt", truncation=True, max_length=2048
        )
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=512,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
            )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract assistant response
        if "assistant:" in response:
            response = response.split("assistant:")[-1].strip()

        return response

    def _extract_choices(self, response: str) -> List[Dict]:
        """Extract choice options from generated text (simplified)"""
        # Simple pattern matching for choices
        import re

        choices = []
        choice_patterns = [
            r"[1-9]\.\s*(.+?)(?=\n[1-9]\.|$)",
            r"選項[一二三四五][:：]\s*(.+?)(?=\n選項|$)",
            r"[A-D]\)\s*(.+?)(?=\n[A-D]\)|$)",
        ]

        for pattern in choice_patterns:
            matches = re.findall(pattern, response, re.MULTILINE | re.DOTALL)
            if matches:
                for i, match in enumerate(matches[:4]):  # Max 4 choices
                    choices.append(
                        {
                            "id": f"choice_{i+1}",
                            "text": match.strip(),
                            "risk_level": random.choice(["low", "medium", "high"]),
                        }
                    )
                break

        # Fallback: create default choices
        if not choices:
            choices = [
                {"id": "choice_1", "text": "謹慎觀察", "risk_level": "low"},
                {"id": "choice_2", "text": "主動行動", "risk_level": "medium"},
            ]

        return choices

In [ ]:
# === Cell 6: Context-Aware Generation ===
class ContextAwareGenerator:
    """Enhanced generator with better context awareness"""

    def __init__(self, event_generator: EventGenerator):
        self.generator = event_generator
        self.recent_events = []  # Track recent events for continuity

    def generate_contextual_event(
        self, event_type: EventType, player_state: Dict, game_history: List[Dict] = None
    ) -> Dict:
        """Generate event with full game context"""

        # Build enhanced context query
        context_elements = [player_state.get("location", ""), event_type.value]

        # Add recent events for continuity
        if game_history:
            recent_actions = game_history[-3:]  # Last 3 actions
            for action in recent_actions:
                if "choice_made" in action:
                    context_elements.append(action["choice_made"])

        context_query = " ".join(context_elements)

        # Generate event
        event = self.generator.generate_event(event_type, player_state, context_query)

        # Add continuity hints
        if self.recent_events:
            event["continuity_note"] = self._generate_continuity_note()

        # Track this event
        self.recent_events.append(
            {
                "type": event_type.value,
                "location": player_state.get("location"),
                "timestamp": len(self.recent_events),
            }
        )

        # Keep only recent events
        if len(self.recent_events) > 5:
            self.recent_events.pop(0)

        return event

    def _generate_continuity_note(self) -> str:
        """Generate note about story continuity"""
        if len(self.recent_events) >= 2:
            return f"與先前的{self.recent_events[-1]['type']}事件相關"
        return ""

In [ ]:
# === Cell 7: Citation & Source Tracking ===
class CitationManager:
    """Manage citations and source tracking for events"""

    @staticmethod
    def format_citations(event: Dict) -> str:
        """Format citations for display"""
        if not event.get("sources"):
            return ""

        citation_text = "\n\n📚 **參考來源：**\n"
        for i, source in enumerate(event["sources"], 1):
            citation_text += f"[{i}] {source}\n"

        return citation_text

    @staticmethod
    def verify_groundedness(event_text: str, context: str) -> float:
        """Simple groundedness check"""
        if not context or not event_text:
            return 0.0

        # Simple keyword overlap
        context_words = set(context.lower().split())
        event_words = set(event_text.lower().split())

        if len(context_words) == 0:
            return 0.0

        overlap = len(context_words.intersection(event_words))
        return overlap / len(context_words)

In [ ]:
# === Cell 8: Difficulty Scaling ===
class DifficultyScaler:
    """Dynamic difficulty adjustment for events"""

    @staticmethod
    def calculate_player_power(player_state: Dict) -> float:
        """Calculate player's current power level"""
        hp_ratio = player_state.get("hp", 100) / player_state.get("max_hp", 100)
        skill_count = len(player_state.get("skills", []))
        item_count = len(player_state.get("inventory", []))

        # Simple power calculation
        power = (hp_ratio * 0.5) + (skill_count * 0.3) + (item_count * 0.2)
        return min(power, 2.0)  # Cap at 2.0

    @staticmethod
    def adjust_event_difficulty(event: Dict, player_power: float) -> Dict:
        """Adjust event difficulty based on player power"""
        base_difficulty = event.get("difficulty", 2)

        # Adjust based on player power
        if player_power < 0.5:
            adjusted_difficulty = max(1, base_difficulty - 1)
        elif player_power > 1.5:
            adjusted_difficulty = min(5, base_difficulty + 1)
        else:
            adjusted_difficulty = base_difficulty

        event["difficulty"] = adjusted_difficulty
        event["player_power"] = player_power

        return event

In [ ]:
# === Cell 9: Smoke Test ===
print("=== Event Generation Smoke Test ===")

# Test player state
test_player = {
    "location": "古老的森林",
    "hp": 75,
    "max_hp": 100,
    "inventory": ["生鏽的劍", "治療藥水"],
    "skills": ["劍術", "潛行"],
    "investigation": 2,
}

# Initialize components
if world_kb.index is not None:
    event_gen = EventGenerator(world_kb)
    context_gen = ContextAwareGenerator(event_gen)

    # Generate exploration event
    exploration_event = context_gen.generate_contextual_event(
        EventType.EXPLORATION, test_player
    )

    print("🎮 Generated Exploration Event:")
    print(f"ID: {exploration_event['event_id']}")
    print(f"Type: {exploration_event['type']}")
    print(f"Difficulty: {exploration_event['difficulty']}")
    print(f"\nDescription:\n{exploration_event['description']}")
    print(f"\nChoices:")
    for choice in exploration_event["choices"]:
        print(f"- {choice['text']} (風險: {choice['risk_level']})")

    # Add citations
    citations = CitationManager.format_citations(exploration_event)
    if citations:
        print(citations)

    # Calculate groundedness
    groundedness = CitationManager.verify_groundedness(
        exploration_event["description"], exploration_event["context_used"]
    )
    print(f"\n📊 Groundedness Score: {groundedness:.2f}")

    print("\n✅ Event generation successful!")

else:
    print("⚠️ World KB not available, creating mock event...")

    # Mock event for testing without KB
    mock_event = {
        "event_id": "exploration_1234",
        "type": "exploration",
        "description": "你進入了一片神秘的森林，古老的樹木遮天蔽日，微弱的陽光透過樹葉間隙灑下。遠處傳來不明的聲響...",
        "choices": [
            {
                "id": "choice_1",
                "text": "小心翼翼地前進調查聲音來源",
                "risk_level": "medium",
            },
            {"id": "choice_2", "text": "在原地觀察等待", "risk_level": "low"},
            {"id": "choice_3", "text": "大聲呼喊看看是否有回應", "risk_level": "high"},
        ],
        "difficulty": 2,
        "sources": ["mock_forest_lore"],
        "context_used": "森林設定：古老神秘，有未知的生物...",
    }

    print("🎮 Mock Exploration Event:")
    print(f"Description: {mock_event['description']}")
    print("Choices:")
    for choice in mock_event["choices"]:
        print(f"- {choice['text']} (風險: {choice['risk_level']})")

    print("\n✅ Mock event generation successful!")

In [ ]:
# === Cell 10: Usage Examples ===
print("\n=== Usage Examples ===")

def demo_event_pipeline():
    """Demonstrate complete event generation pipeline"""

    # Different game scenarios
    scenarios = [
        {
            "name": "健康探險者",
            "state": {"location": "山洞入口", "hp": 90, "max_hp": 100, "inventory": ["火把", "繩索"], "skills": ["攀爬"]}
        },
        {
            "name": "受傷的戰士",
            "state": {"location": "廢棄城堡", "hp": 30, "max_hp": 100, "inventory": ["破損盾牌"], "skills": ["戰鬥"]}
        },
        {
            "name": "新手冒險者",
            "state": {"location": "村莊邊緣", "hp": 100, "max_hp": 100, "inventory": [], "skills": []}
        }
    ]

    for scenario in scenarios:
        print(f"\n--- {scenario['name']} ---")

        # Calculate player power
        power = DifficultyScaler.calculate_player_power(scenario['state'])
        print(f"Player Power: {power:.2f}")

        # Mock event generation (since we may not have world KB)
        event_types = [EventType.EXPLORATION, EventType.ENCOUNTER, EventType.DISCOVERY]
        selected_type = random.choice(event_types)

        print(f"Event Type: {selected_type.value}")
        print(f"Location: {scenario['state']['location']}")
        print(f"HP: {scenario['state']['hp']}/{scenario['state']['max_hp']}")

        # Show how difficulty would be adjusted
        base_difficulty = random.randint(2, 4)
        if power < 0.5:
            adjusted = max(1, base_difficulty - 1)
        elif power > 1.5:
            adjusted = min(5, base_difficulty + 1)
        else:
            adjusted = base_difficulty

        print(f"Difficulty: {base_difficulty} → {adjusted} (adjusted for player power)")

demo_event_pipeline()

print("\n=== Key Parameters ===")
print("Low VRAM Options:")
print("- load_in_4bit=True (reduce model memory)")
print("- max_new_tokens=512 (limit generation)")
print("- batch_size=1 (process one event at a time)")
print("- Use smaller embedding models like bge-small-zh-v1.5")

print("\nEvent Template Configuration:")
print("- Difficulty ranges: 1-5 scale")
print("- Choice limits: 2-4 options per event")
print("- Context window: ~200 chars from KB")
print("- Temperature: 0.7 for creative but controlled generation")

print("\n=== When to Use This ===")
print("✅ Text adventure games needing dynamic content")
print("✅ Educational scenarios with knowledge base backing")
print("✅ Interactive storytelling with citation requirements")
print("✅ Games requiring consistent world-building")
print("❌ Simple random events without context needs")
print("❌ Real-time action games (due to generation latency)")